# OSU Small Animals 
[OSU Small Animals](https://lila.science/datasets/ohio-small-animals/) is a benchmark dataset, a static collection of images used to evaluate the capability of various ML approaches, developed from terrestrial camera trap images. This notebook contains pseudocode giving you an idea of what to do as you work to training a classifer on this labeled data. 

Unlike previous notebooks, this one is very much a skeleton with some extra information to frame your workflow. In other words, you will write most of the code yourself. You can write everything from scratch or treat this as a vibe coding session (i.e. asking your favorite LLM for help). Please ask the instructors lots of questions. 

## Common Objects in COntext format
[Common Objects in COntext (COCO)](https://link.springer.com/chapter/10.1007/978-3-319-10602-1_48) is one of the classic machine learning benchmark data (62000 citations and counting!). The material below explaining the COCO format is adapted from the [FathomNet Python API Tutorial](https://github.com/fathomnet/fathomnet-py/blob/main/examples/tutorial.ipynb). 

The original COCO dataset is a large annotated image dataset containing bounding boxes and segmentation masks for 91 categories in 100s of thousands of images. The dataset led to lots of novel AI models and the format has become standard for many python tools. Widely used architectures like Pytorch (what we've been using in this workshop) and [YOLO](https://docs.ultralytics.com/tasks/classify/) have a [prebuilt DataLoader object for COCO](https://docs.pytorch.org/vision/stable/_modules/torchvision/datasets/coco.html). COCO format supports [multiple types of annotations](https://cocodataset.org/#overview) including: classification, bounding box, segmantions, and keypoints. Regardless of type, annotations are organized into a standard format to make it easier to work with. Typically, the annotation files are distributed as json serialized nested dictionaries. The highest level looks like this:

```
{
  "info": info,
  "images": [image],
  "categories": [category]
  "annotations": [annotation]
}
```

Each field contains inter-related pieces of information for your model training code to use. `info` is a dictionary that contains some metadata about the entire dataset:

```
info = {
    "year": 2023,
    "version": "0",
    "description": "Generated by FathomNet",
    "contributor": "FathomNet",
    "url": "https://fathomnet.org",
    "date_created": "2023/02/23"
}
```

The `images` field consists of a list of `image` objects that specify the file name, image dimensions, permanent url, etc. 

```
image = {
    "id": 1,
    "width": 1920,
    "height": 1080,
    "file_name": "754e6a28-a8eb-4cb3-a0b9-3f2d5daacbae.png",
    "license": 0,
    "flickr_url": "https://fathomnet.org/static/m3/staging/Doc%20Ricketts/images/0861/00_12_12_05.png",
    "coco_url": "https://fathomnet.org/static/m3/staging/Doc%20Ricketts/images/0861/00_12_12_05.png",
    "date_captured": "2016-06-16 00:00:00"
}
```

The `categories` field is a list of `category` objects organized by numeric ids.

```
category = {
  "id": 2,
  "name": "Actinernus",
  "supercategory": ""
}
```

Finally, `annotations` is a list of `annotation` objects that bring it all together. 

```
annotation = {
      "id": 1,
      "image_id": 1,
      "category_id": 2,
      "segmentation": [],
      "area": 51200.0,
      "bbox": [
        200.0,
        433.0,
        256.0,
        200.0
      ],
      "iscrowd": 0
}
```

Note that the `id` fields are specific to the list of objects. The example `annotation` object tells us that in the image associated with `image_id` 1, there a bounding box located at position `[200, 433]` with a width of `256` pixels and a heigh of `200` pixels. The label associated with that bounding box is `category_id` 2 which corresponds the anemone "Actinernus". If there are not bounding boxes associated with the annotation, the `bbox` and `area` fields are empty. 

## Explore OSU Small Animals
Start by exploring the OSU Small Animals dataset by loading it in, trying to view images, and exploring metadata. The data can be found in `/groups/cv-workshop/ohio_small_animals` with the images in the `Images` subdirectory. The image and annotation data is in the COCO formated json file `osu-small-animals.json`.

This dataset is distributed in a modified version of COCO called [COCO Camera Traps](https://lila.science/coco-camera-traps) (CCT) format. CCT is a superset of COCO and is compatible with tools that expect COCO-formatted data. It includes additional metadata relevant to camera trap deployments, namely location. The location field is in the image field described above and can be useful for making training and validation datasets.  

In [1]:
import os
import json
import random
import pandas as pd

from tqdm.notebook import tqdm
from collections import defaultdict

from megadetector.utils.ct_utils import is_list_sorted
from megadetector.utils.ct_utils import sort_dictionary_by_value
from megadetector.utils.path_utils import parallel_copy_files
from megadetector.utils.path_utils import recursive_file_list
from megadetector.utils.split_locations_into_train_val import split_locations_into_train_val
from megadetector.visualization.visualization_utils import resize_image_folder

DATA_FOLDER = '/groups/cv-workshop/ohio_small_animals'

In [2]:
metadata_file = os.path.join(DATA_FOLDER, 'osu-small-animals.json')

with open(metadata_file,'r') as f:
    image_info = json.load(f)

In [3]:
print('Loaded information for {} images'.format(len(image_info['images'])))

Loaded information for 118554 images


Print out the counts per category.

In [4]:
image_id_to_annotations = defaultdict(list)

category_id_to_name = {c['id']:c['name'] for c in image_info['categories']}
category_name_to_id = {c['name']:c['id'] for c in image_info['categories']}

category_name_to_count = defaultdict(int)

for ann in image_info['annotations']:
    image_id_to_annotations[ann['image_id']].append(ann)

image_id_to_categories = defaultdict(set)

for im in tqdm(image_info['images']):

    annotations_this_image = image_id_to_annotations[im['id']]

    if len(annotations_this_image) == 0:
        image_id_to_categories[im['id']].add('unannotated')        
    else:
        for ann in annotations_this_image:
            category_name = category_id_to_name[ann['category_id']]
            image_id_to_categories[ann['image_id']].add(category_name)
            category_name_to_count[category_name] += 1

category_name_to_count = sort_dictionary_by_value(category_name_to_count,reverse=True)

for category_name in category_name_to_count:
    print('{}: {}'.format(category_name,category_name_to_count[category_name]))


  0%|          | 0/118554 [00:00<?, ?it/s]

eastern_gartersnake: 31899
song_sparrow: 14567
meadow_vole: 14169
empty: 11448
white-footed_mouse: 10548
northern_house_wren: 5934
invertebrate: 5075
common_five-lined_skink: 5045
masked_shrew: 4242
eastern_cottontail: 3263
long-tailed_weasel: 2325
woodland_jumping_mouse: 1510
plains_gartersnake: 1272
eastern_massasauga: 1189
virginia_opossum: 985
common_yellowthroat: 802
n._short-tailed_shrew: 746
dekay's_brownsnake: 529
american_mink: 425
american_toad: 340
eastern_racer_snake: 293
smooth_greensnake: 264
eastern_chipmunk: 198
northern_leopard_frog: 193
meadow_jumping_mouse: 160
butler's_gartersnake: 155
eastern_ribbonsnake: 133
northern_watersnake: 121
star-nosed_mole: 111
striped_skunk: 104
eastern_milksnake: 72
gray_ratsnake: 68
eastern_hog-nosed_snake: 67
raccoon: 62
green_frog: 47
woodchuck: 44
kirtland's_snake: 44
indigo_bunting: 23
painted_turtle: 23
sora: 13
american_bullfrog: 12
gray_catbird: 10
red-bellied_snake: 9
brown_rat: 8
snapping_turtle: 6
eastern_bluebird: 1


Select categories that have more than 100 labeled examples

In [5]:
raw_category_to_target_category = {}

min_images_to_include_in_training = 100

for raw_category_name in category_name_to_count:
    if category_name_to_count[raw_category_name] > min_images_to_include_in_training:
        raw_category_to_target_category[raw_category_name] = raw_category_name
    else:
        print('Excluding category {}'.format(raw_category_name))

Excluding category eastern_milksnake
Excluding category gray_ratsnake
Excluding category eastern_hog-nosed_snake
Excluding category raccoon
Excluding category green_frog
Excluding category woodchuck
Excluding category kirtland's_snake
Excluding category indigo_bunting
Excluding category painted_turtle
Excluding category sora
Excluding category american_bullfrog
Excluding category gray_catbird
Excluding category red-bellied_snake
Excluding category brown_rat
Excluding category snapping_turtle
Excluding category eastern_bluebird


These COCO annotation files also have location metadata. Count the animals from each location.

In [6]:
location_to_category_counts = {}
category_to_location_counts = {}

all_locations = set()

for im in image_info['images']:

    location = im['location']
    all_locations.add(location)
    categories_this_image = image_id_to_categories[im['id']]

    if location not in location_to_category_counts:
        location_to_category_counts[location] = {}

    for raw_category_name in categories_this_image:

        if raw_category_name not in raw_category_to_target_category:
            continue
        category_name = raw_category_to_target_category[raw_category_name]

        if category_name not in category_to_location_counts:
            category_to_location_counts[category_name] = {}
        
        if location not in category_to_location_counts[category_name]:
            category_to_location_counts[category_name][location] = 0

        if category_name not in location_to_category_counts[location]:
            location_to_category_counts[location][category_name] = 0
            
        location_to_category_counts[location][category_name] = \
            location_to_category_counts[location][category_name] + 1
    
        category_to_location_counts[category_name][location] = \
            category_to_location_counts[category_name][location] + 1


## Create training and validation
In earlier notebooks, you were given premade training and validation datasets. Recall, the training data is used for tuning your model and the validation is held out for testing. Come up with a training and validation split for your model. This could be purely random or based on some relevant metadata. Up to you!

In [9]:
# split on location

train_fraction = 0.85

train_on_crops = False
random.seed(0)

val_locations,category_to_val_fraction = \
      split_locations_into_train_val(location_to_category_counts,
                                     n_random_seeds=10000,
                                     target_val_fraction=1.0-train_fraction,
                                     category_to_max_allowable_error=None,                                   
                                     category_to_error_weight=None,
                                     default_max_allowable_error=0.2,
                                     require_complete_coverage=True)

print('Assigned {} of {} locations to the val split'.format(len(val_locations),len(all_locations)))

Splitting 30 categories over 168 locations


100%|██████████| 10000/10000 [00:05<00:00, 1668.04it/s]


47 of 10000 random seeds satisfied hard constraints

Val locations:

DORRS
MONC295N
GRN2
KILC3W
KILD1E
KILC5WA
GNR1
FCM2
MLN2B
KILQ1S
CBNP1S
FCP1
KILJ14W
KILB11W
KPC3
KILH10S
KILQ3S
KILC1E
RMW1
KILD1W
KILF10S
KPA2
KILE1W
WIL2
KILC2S

Val fractions by category:

eastern_gartersnake (31899) 0.12
song_sparrow (14567) 0.09
meadow_vole (14169) 0.08
empty (11448) 0.10
white-footed_mouse (10548) 0.17
northern_house_wren (5934) 0.21
invertebrate (5075) 0.11
common_five-lined_skink (5045) 0.07
masked_shrew (4242) 0.17
eastern_cottontail (3263) 0.17
long-tailed_weasel (2325) 0.14
woodland_jumping_mouse (1510) 0.17
plains_gartersnake (1272) 0.07
eastern_massasauga (1189) 0.15
virginia_opossum (985) 0.11
common_yellowthroat (802) 0.27
n._short-tailed_shrew (746) 0.13
dekay's_brownsnake (529) 0.17
american_mink (425) 0.03
american_toad (340) 0.10
eastern_racer_snake (293) 0.13
smooth_greensnake (264) 0.10
eastern_chipmunk (198) 0.12
northern_leopard_frog (193) 0.12
meadow_jumping_mouse (160) 0.25


In [12]:
# write these splits out
ptf = '/noc/users/ericor/computer-vision-workshop/object_detection'

train_locations = []
for location in all_locations:
    if location not in val_locations:
        train_locations.append(location)

assert len(train_locations) + len(val_locations) == len(all_locations)

split_file = os.path.join(ptf,'location_splits.json')

split_info = {}
split_info['train'] = sorted(list(train_locations))
split_info['val'] = sorted(list(val_locations))

with open(split_file,'w') as f:
    json.dump(split_info,f,indent=1)

In [13]:
# Assign images to unique categories
category_name_to_images = defaultdict(list)
images_with_multiple_categories = []
images_with_ignored_categories = []

for im in image_info['images']:

    categories_this_image = image_id_to_categories[im['id']]
    assert len(categories_this_image) > 0
    if len(categories_this_image) > 1:
        images_with_multiple_categories.append([im['file_name'],categories_this_image])
        continue
    category_name = list(categories_this_image)[0]
    if category_name not in raw_category_to_target_category:
        images_with_ignored_categories.append(im['file_name'])
        continue
    category_name_to_images[category_name].append(im)

print('Ignored {} images with multiple categories'.format(len(images_with_multiple_categories)))   
print('Ignored {} images with excluded categories'.format(len(images_with_ignored_categories)))

print('Before sampling:\n')
for category_name in category_name_to_images:
    print('{}: {}'.format(category_name,
                          len(category_name_to_images[category_name])))

Ignored 0 images with multiple categories
Ignored 509 images with excluded categories
Before sampling:

empty: 11448
american_toad: 340
northern_leopard_frog: 193
common_yellowthroat: 802
northern_house_wren: 5934
song_sparrow: 14567
invertebrate: 5075
common_five-lined_skink: 5045
american_mink: 425
eastern_chipmunk: 198
eastern_cottontail: 3263
long-tailed_weasel: 2325
masked_shrew: 4242
meadow_jumping_mouse: 160
meadow_vole: 14169
n._short-tailed_shrew: 746
star-nosed_mole: 111
striped_skunk: 104
virginia_opossum: 985
white-footed_mouse: 10548
woodland_jumping_mouse: 1510
butler's_gartersnake: 155
dekay's_brownsnake: 529
eastern_gartersnake: 31899
eastern_massasauga: 1189
eastern_racer_snake: 293
eastern_ribbonsnake: 133
northern_watersnake: 121
plains_gartersnake: 1272
smooth_greensnake: 264


In [14]:
# enforce a max of blank samples
# max_blanks = 20000
max_blanks = 200000

if len(category_name_to_images['empty']) > max_blanks:
    category_name_to_images['empty'] = random.sample(category_name_to_images['empty'],k=max_blanks)

print('After sampling:\n')
for category_name in category_name_to_images:
    print('{}: {}'.format(category_name,
                          len(category_name_to_images[category_name])))
    

After sampling:

empty: 11448
american_toad: 340
northern_leopard_frog: 193
common_yellowthroat: 802
northern_house_wren: 5934
song_sparrow: 14567
invertebrate: 5075
common_five-lined_skink: 5045
american_mink: 425
eastern_chipmunk: 198
eastern_cottontail: 3263
long-tailed_weasel: 2325
masked_shrew: 4242
meadow_jumping_mouse: 160
meadow_vole: 14169
n._short-tailed_shrew: 746
star-nosed_mole: 111
striped_skunk: 104
virginia_opossum: 985
white-footed_mouse: 10548
woodland_jumping_mouse: 1510
butler's_gartersnake: 155
dekay's_brownsnake: 529
eastern_gartersnake: 31899
eastern_massasauga: 1189
eastern_racer_snake: 293
eastern_ribbonsnake: 133
northern_watersnake: 121
plains_gartersnake: 1272
smooth_greensnake: 264


## Train your model
Train a classification model based on the COCO formatted data. You can try to write a custom DataLoader to read in COCO data for classification (as opposed to object detection). Or you could try a different architecture all together. [Ultralytics](https://docs.ultralytics.com/tasks/classify/#models), for example, distributes classification models that can read in data from COCO formatted metadata (NB: this is a different tool from Pytorch and will require installing a new package). 